In [1]:
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import keras
from efficientnet_lite import EfficientNetLiteB0
from keras.optimizers import Adam
from tensorflow.keras.layers.experimental import preprocessing
from IPython.display import clear_output
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
from keras.callbacks import EarlyStopping

from keras.applications.vgg16 import VGG16

In [14]:
data_augmentation = keras.Sequential(
    [
        # preprocessing.RandomZoom(height_factor=0.2),
        preprocessing.RandomRotation(0.15)
    ]
)

backbone = EfficientNetLiteB0(
   weights=None, 
   input_shape=(128,128, 3),
   include_top=False
)

model = tf.keras.Sequential([
    backbone,
    Flatten(),
    Dense(3, activation = 'softmax')  # 2 = num classes
])

model.summary()
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=3, restore_best_weights=True)
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetlite0 (Function  (None, 4, 4, 1280)       3413024   
 al)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20480)             0         
                                                                 
 dense_1 (Dense)             (None, 3)                 61443     
                                                                 
Total params: 3,474,467
Trainable params: 3,432,451
Non-trainable params: 42,016
_________________________________________________________________


In [28]:
clock = 0
video_capture = cv2.VideoCapture(0)
dir_save = r"C:\Users\Jerry\OneDrive\Documents\Imperial\3rd year\3rd year Group project\Image\Train data\Faces\%d.jpeg" # Directory folder to store user facial data

while True:
    success, frame = video_capture.read()  # read frames from the video
    if not success:
        break

    if clock % 100 == 0:
        clear_output()
        frame.resize([128, 128, 3])
        frame = frame / 255.
        score_raw = model.predict(frame[None])[0]
        print(score_raw)
        score = tf.math.argmax(score_raw)
    if score == 1:
        cv2.putText(frame, 'Hi boss!', (50, 50), cv2.LINE_AA, fontScale = 1, color = (255, 0, 0))
    elif score == 0:
        cv2.putText(frame, 'Anyone there?', (50, 50), cv2.LINE_AA, fontScale = 1, color = (255, 0, 0))

    cv2.imshow("Test", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("z"):
        for i in range(20):
            success, image = video_capture.read()
            if not success:
                break
            cv2.putText(frame, 'Taking pictures...', (50, 50), cv2.LINE_AA, fontScale = 1, color = (255, 0, 0))
            cv2.imshow("Test", frame)
            cv2.imwrite(dir_save % (81 + i), image)
        print('Done.')

    elif cv2.waitKey(1) & 0xFF == ord("q"):
        break

    clock += 1

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 39ms/step
[9.9992740e-01 5.1951648e-14 7.2625087e-05]


In [ ]:
dir = r'C:\Users\Jerry\OneDrive\Documents\Imperial\3rd year\3rd year Group project\Image\Train data\Faces' # Directory folder which stores all facial data
data = tf.keras.preprocessing.image_dataset_from_directory(dir, image_size=(128, 128), batch_size = 300, label_mode = 'categorical')

In [ ]:
for images, labels in data.take(1):  # Takes the 1st batch; in our case there's only 1 batch
    train_X = images.numpy() / 255.
    labels = labels.numpy()

labels = to_categorical(labels, 3)
print(labels[5])
plt.imshow(train_X[5])

In [32]:
model.fit(train_X, labels, epochs=1, batch_size = 1, validation_split = 0.1)

270/270 [==============================] - 26s 78ms/step - loss: 0.1489 - categorical_accuracy: 0.9704 - val_loss: 0.7665 - val_categorical_accuracy: 0.8333


In [ ]:
score_raw = model.predict(train_X[0][None])[0]
score = tf.math.argmax(score_raw)
plt.imshow(train_X[0])
print(score)